# PyChe Tutorial: Run + Full Diagnostics

This notebook shows in-memory analysis from `res.mod` and `res.fis`, file-based diagnostics, and MPI launch commands.

In [ ]:
from pyche import GCEModel, create_diagnostic_plots, read_outputs
from pyche.diagnostics import diagnostics_from_tables
import numpy as np
import matplotlib.pyplot as plt


## 1) In-memory run

In [ ]:
m = GCEModel()
res = m.MinGCE(
    endoftime=13700,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=True,
    backend='auto',
    output_mode='dataframe',
    write_output=False,
    return_results=True,
)
res.mod.shape, res.fis.shape


## 1b) Same run without approximations (baseline settings)

In [ ]:
m = GCEModel()
res_noapprox = m.MinGCE(
    endoftime=13700,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=True,
    backend='auto',
    output_mode='dataframe',
    write_output=False,
    return_results=True,
    adaptive_timestep=False,
    interp_cache=False,
    interp_cache_guard=False,
    profile_timing=False,
    spalla_stride=1,
    spalla_inactive_threshold=0.0,
    spalla_lut=False,
)
res_noapprox.mod.shape, res_noapprox.fis.shape


## 2) Diagnostics from in-memory arrays

In [ ]:
diag = diagnostics_from_tables(res.mod, res.fis)
diag


In [ ]:
mod_cols = {name: i for i, name in enumerate(res.mod_columns)}
fis_cols = {name: i for i, name in enumerate(res.fis_columns)}

time_f = res.fis[:, fis_cols['time']]
sfr = res.fis[:, fis_cols['sfr']]
allv = res.fis[:, fis_cols['all']]
gas = res.fis[:, fis_cols['gas']]
stars = res.fis[:, fis_cols['stars']]
remn = res.fis[:, fis_cols['remn']]
zeta = res.fis[:, fis_cols['zeta']]

time_m = res.mod[:, mod_cols['time']]
fe = res.mod[:, mod_cols['Fe']]
h = res.mod[:, mod_cols['H']]
o16 = res.mod[:, mod_cols['O16']]
mg = res.mod[:, mod_cols['Mg']]

eps = 1.0e-30
FE_H_SUN_MASS = 56.0 * 10.0 ** (-4.50)
O_FE_SUN_MASS = (16.0 / 56.0) * 10.0 ** (8.69 - 7.50)
MG_FE_SUN_MASS = (24.305 / 56.0) * 10.0 ** (7.60 - 7.50)
log_feh = np.log10(np.maximum(fe / np.maximum(h, eps), eps) / FE_H_SUN_MASS)
log_o_fe = np.log10(np.maximum(o16 / np.maximum(fe, eps), eps) / O_FE_SUN_MASS)
log_mg_fe = np.log10(np.maximum(mg / np.maximum(fe, eps), eps) / MG_FE_SUN_MASS)

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(time_f, sfr)
ax[0].set_title('SFR vs Time')
ax[1].plot(time_f, zeta)
ax[1].set_title('Zeta vs Time')
plt.tight_layout()

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(time_f, allv, label='all')
ax.plot(time_f, gas, label='gas')
ax.plot(time_f, stars, label='stars')
ax.plot(time_f, remn, label='remn')
ax.set_title('Mass Budget Evolution')
ax.legend(frameon=False)
plt.tight_layout()

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(time_m, log_feh, lw=1.4)
ax.set_title('[Fe/H] vs Time')
ax.set_ylim(-7,0.8)
plt.tight_layout()

mask = np.isfinite(log_feh)
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(log_feh[mask], log_o_fe[mask], label='[O/Fe]', lw=1.3)
ax.plot(log_feh[mask], log_mg_fe[mask], label='[Mg/Fe]', lw=1.3)
ax.set_title('Abundance Tracks')
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('[X/Fe]')
ax.set_xlim(-5,1)
ax.set_ylim(-0.1,0.8)
ax.legend(frameon=False)
plt.tight_layout()

stars_delta = np.diff(stars, prepend=stars[0])
stars_delta = np.maximum(stars_delta, 0.0)
mdf_mask = np.isfinite(log_feh) & (log_feh >= -5.0) & (log_feh <= 1.0)
fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(log_feh[mdf_mask], bins=np.linspace(-5, 1, 61), weights=stars_delta[mdf_mask], alpha=0.75)
ax.set_title('MDF [Fe/H]')
ax.set_xlabel('[Fe/H]')
plt.tight_layout()


## 3) Save a run, load it back, and create plot files

In [ ]:
out_dir = 'RISULTATI_PYCHE_NOTEBOOK'
m.MinGCE(
    endoftime=1000,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=False,
    output_dir=out_dir,
    output_mode='dataframe',
    df_binary_format='pickle',
    write_output=True,
    return_results=False,
)
payload = read_outputs(out_dir, prefer='dataframe', binary_format='pickle')
payload['mod'].shape, payload['fis'].shape, payload['format']


In [ ]:
paths = create_diagnostic_plots(out_dir, prefer='dataframe', binary_format='pickle')
paths


## 4) MPI run example with 4 ranks

Run from shell (outside notebook):

```bash
mpiexec -n 4 python -c "from pyche import GCEModel; m=GCEModel(); m.MinGCE(13700,3000.0,50.0,0.3,0.0,10000,10000,use_mpi=True,show_progress=False,backend='auto',output_dir='RISULTATI_MPI4',output_mode='dataframe',df_binary_format='pickle')"
```

With Cython backend (compile first):

```bash
pip install cython
python setup.py build_ext --inplace
mpiexec -n 4 python -c "from pyche import GCEModel; m=GCEModel(); m.MinGCE(13700,3000.0,50.0,0.3,0.0,10000,10000,use_mpi=True,show_progress=False,backend='cython',output_dir='RISULTATI_MPI4_CY',output_mode='dataframe',df_binary_format='pickle')"
```